# BB84 Quantum Key Distribution (QKD) Protocol using Qiskit


In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
# Importing standard Qiskit libraries
from qiskit import QuantumCircuit, execute
from qiskit.providers.aer import QasmSimulator
from qiskit.visualization import *

## Choosing bases and encoding states

Alice generates two binary strings. One encodes the basis for each qubit:

$0 \rightarrow$ Computational basis

$1  \rightarrow$ Hadamard basis

The other encodes the state:

$0  \rightarrow|0\rangle$ or $|+\rangle $ 

$1  \rightarrow|1\rangle$  or  $|-\rangle $ 

Bob also generates a binary string and uses the same convention to choose a basis for measurement


In [2]:
num_qubits = 32

alice_basis = np.random.randint(2, size=num_qubits)
alice_state = np.random.randint(2, size=num_qubits)
bob_basis = np.random.randint(2, size=num_qubits)


print(f"Alice's State:\t {alice_state}")
print(f"Alice's Bases:\t {alice_basis}")
print(f"Bob's Bases:\t {bob_basis}")

Alice's State:	 [1 0 0 0 1 1 0 1 0 1 0 0 1 0 1 1 1 0 1 1 0 0 1 0 0 1 0 0 0 1 1 1]
Alice's Bases:	 [1 0 0 0 0 0 1 1 1 0 1 1 1 1 0 1 1 0 0 1 1 1 0 0 0 0 0 1 1 0 0 0]
Bob's Bases:	 [0 0 1 1 1 1 1 0 1 0 0 1 1 1 0 0 1 0 1 0 1 1 0 1 1 1 1 1 0 1 1 1]


## Creating the circuit

Based on the following results:

$X|0\rangle = |1\rangle$

$H|0\rangle = |+\rangle$

$ HX|0\rangle = |-\rangle$

Our algorithm to construct the circuit is as follows:

1. Whenever Alice wants to encode 1 in a qubit, she applies an $X$ gate to the qubit. To encode 0, no action is needed.
2. Wherever she wants to encode it in the Hadamard basis, she applies an $H$ gate. No action is necessary to encode a qubit in the computational basis.

3. She then _sends_ the qubits to Bob (symbolically represented in this circuit using wires)

4. Bob measures the qubits according to his binary string. To measure a qubit in the Hadamard basis, he applies an $H$ gate to the corresponding qubit and then performs a mesurement on the computational basis. 



In [3]:
bb84_circ = QuantumCircuit(num_qubits)

# Alice prepares her qubits
for index in range(len(alice_basis)):
    if alice_state[index] == 1:
        bb84_circ.x(index)
    if alice_basis[index] == 1:
        bb84_circ.h(index)
bb84_circ.barrier()  

# Bob measures the received qubits
for index in range(len(bob_basis)):
    if bob_basis[index] == 1:
        bb84_circ.h(index)
        
bb84_circ.barrier()        
bb84_circ.measure_all()
    
            
# bb84_circ.draw('mpl')


Now we will run this simulation experiment on the `QasmSimulator()` backend.

In [5]:
job = execute(bb84_circ.reverse_bits(),backend=QasmSimulator(),shots=10)
counts = job.result().get_counts()

# Prints out all the measured binary strings by Bob (reversed)
# for count in [*counts]:
#     print(count) 

In [4]:
counts = execute(bb84_circ.reverse_bits(),backend=QasmSimulator(),shots=10).result().get_counts()
keys = []
for temp_key in counts:
    key = ''
    for i in range(num_qubits):
        if alice_basis[i] == bob_basis[i]: # Only choose bits where Alice and Bob chose the same basis
             key += str(temp_key[i])
    keys.append(key)
print(f'The length of the key is {len(keys[0])}')
print(f"The key contains {(keys[0]).count('0')} zeroes and {(keys[0]).count('1')} ones")

The length of the key is 14
The key contains 9 zeroes and 5 ones


In [5]:
for key in keys:
    print(key)

00010101100010
00010101100010
00010101100010
00010101100010
00010101100010
00010101100010
00010101100010
00010101100010
00010101100010
00010101100010
